In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy import signal
# pandas doesn't show us all the decimals
pd.options.display.precision = 15

In [ ]:
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
rows = 150000
segments = int(np.floor(train.shape[0] / rows))
segments

In [ ]:
psdata = None
for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:(segment)*rows+rows]
    x = None
    for c in np.arange(0,150000,15000):
        #f, Pxx_den = signal.welch(np.pad(seg.acoustic_data[c:], 256, mode='constant'),1024, nperseg=512)
        f, Pxx_den = signal.welch(seg.acoustic_data[c:],1024, nperseg=512)
        Pxx_den = signal.decimate(Pxx_den[f<90],2)
        f = signal.decimate(f[f<90],2).astype(int)
        if (x is None):
            x = pd.DataFrame(data=Pxx_den.reshape(1,-1),columns=['f_'+str(c)+'_'+str(o) for o in f])
        else:
            x = pd.concat([x,pd.DataFrame(data=Pxx_den.reshape(1,-1),columns=['f_'+str(c)+'_'+str(o) for o in f])],
                          axis=1)

    x['target'] = seg['time_to_failure'].values[-1]
    if(psdata is None):
        psdata = x
    else:
        psdata = pd.concat([psdata,x])

In [ ]:
psdata = psdata.reset_index(drop=True)

In [ ]:
psdata[psdata.target<0.1].target.head()

In [ ]:
psdata[psdata.target>15].target.head()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(psdata.iloc[35][psdata.columns[4:-1]])
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(psdata.iloc[1642][psdata.columns[4:-1]])
plt.show()

In [ ]:
psdata.insert(0,'mn',psdata.loc[:,psdata.columns[:-1]].mean(axis=1))
psdata.insert(0,'std',psdata.loc[:,psdata.columns[:-1]].std(axis=1))
psdata.insert(0,'min',psdata.loc[:,psdata.columns[:-1]].min(axis=1))
psdata.insert(0,'max',psdata.loc[:,psdata.columns[:-1]].max(axis=1))
ss = StandardScaler()
psdata[psdata.columns[:-1]] = ss.fit_transform(psdata[psdata.columns[:-1]])
psdata.head()

In [ ]:
def GP(data):
    return (5.681617 +
            0.0500000007*np.tanh(((np.where(np.maximum((((((data["f_0_71"]) > (data["f_15000_35"]))*1.))), (((((data["f_0_15"]) < (data["f_0_75"]))*1.))))>0, -3.0, (9.64347648620605469) )) * 2.0)) +
            0.0500000007*np.tanh(((np.where(data["f_0_87"]>0, -3.0, np.where((((data["f_105000_7"]) > (((data["f_30000_75"]) / 2.0)))*1.)>0, 9.6666698456, -3.0 ) )) * 2.0)) +
            0.0500000007*np.tanh(((np.where((((((data["f_15000_11"]) < (data["f_0_71"]))*1.)) + ((((data["f_0_15"]) < (data["f_0_75"]))*1.)))>0, -3.0, 9.6666698456 )) * 2.0)) +
            0.0500000007*np.tanh(((7.8888897896) - (((np.where((((data["f_0_15"]) < ((((data["f_0_83"]) + (data["f_0_71"]))/2.0)))*1.)>0, 7.8888897896, data["f_0_11"] )) * 2.0)))) +
            0.0500000007*np.tanh(np.where(((((((np.maximum(((data["f_0_75"])), ((data["f_30000_83"])))) + (data["f_30000_71"]))/2.0)) < (np.tanh((data["f_15000_11"]))))*1.)>0, 7.8888897896, -3.0 )) +
            0.0500000007*np.tanh((((((9.0)) * ((((((((data["f_15000_35"]) > (data["f_0_63"]))*1.)) / 2.0)) - ((((data["f_30000_75"]) > (data["f_0_15"]))*1.)))))) * 2.0)) +
            0.0500000007*np.tanh(np.where((((((data["f_15000_35"]) > (data["f_0_71"]))*1.)) - ((((data["f_0_75"]) > (data["f_0_11"]))*1.)))>0, 9.6666698456, (-1.0*((9.6666698456))) )) +
            0.0500000007*np.tanh((((((((((((data["f_15000_11"]) > (data["f_0_83"]))*1.)) - (np.cos(((((data["max"]) > (data["mn"]))*1.)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((np.where(((((((data["f_15000_71"]) + (data["f_0_75"]))/2.0)) > (data["f_15000_39"]))*1.)>0, -3.0, (((data["f_30000_43"]) < (data["f_30000_15"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh(((np.where((((data["f_30000_11"]) < ((((data["f_15000_71"]) + (data["f_90000_75"]))/2.0)))*1.)>0, -2.0, (((data["f_0_63"]) < (data["f_120000_43"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_45000_35"]) > (data["f_0_47"]))*1.)) - ((((((data["f_45000_75"]) > (np.sin((data["f_0_15"]))))*1.)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((((np.sin((data["f_45000_3"]))) > (((data["f_0_75"]) / 2.0)))*1.)) * 2.0)) - (1.0))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["mn"]) < (data["f_15000_15"]))*1.)) - ((((((data["f_15000_11"]) < (np.maximum(((data["f_0_83"])), ((data["f_0_55"])))))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(((np.where((((data["f_0_75"]) > (data["f_0_15"]))*1.)>0, -3.0, np.cos(((((((data["f_0_71"]) > (data["f_15000_35"]))*1.)) * 2.0))) )) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_0_79"]<0, np.where(((data["f_135000_23"]) + ((((data["f_0_15"]) < (data["f_0_75"]))*1.)))<0, 7.8888897896, -3.0 ), -3.0 )) +
            0.0500000007*np.tanh(((np.where((((data["f_45000_7"]) < ((((data["f_0_87"]) + (data["f_0_83"]))/2.0)))*1.)>0, -3.0, (((data["f_60000_23"]) < (data["f_75000_27"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh(((np.where((((data["f_15000_15"]) < (data["f_0_75"]))*1.)>0, -3.0, np.cos(((((((data["max"]) < (data["mn"]))*1.)) * 2.0))) )) * 2.0)) +
            0.0500000007*np.tanh((-1.0*((np.where(((data["f_45000_15"]) + ((((np.maximum(((data["f_0_63"])), ((data["f_60000_71"])))) > (data["f_45000_15"]))*1.)))>0, 7.8888897896, -3.0 ))))) +
            0.0500000007*np.tanh(np.where(((data["std"]) - (data["f_0_71"]))<0, -3.0, np.where(((data["f_15000_11"]) - (data["f_0_75"]))<0, -3.0, 7.8888897896 ) )) +
            0.0500000007*np.tanh(((np.where(np.maximum(((data["f_15000_83"])), (((((data["f_45000_71"]) > (data["f_15000_15"]))*1.))))>0, -3.0, (((data["f_15000_3"]) > (data["f_0_75"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh(((np.where((((data["f_15000_15"]) < (data["f_15000_75"]))*1.)>0, -3.0, np.cos(((((((data["f_0_71"]) > (data["f_15000_35"]))*1.)) * 2.0))) )) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_0_75"]) < (data["f_0_3"]))*1.)) - ((((((data["f_75000_55"]) > (((data["f_120000_7"]) * 2.0)))*1.)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((4.0)) - (((7.8888897896) * ((((np.maximum((((((data["f_45000_55"]) + (data["f_0_83"]))/2.0))), ((data["f_0_47"])))) > (data["f_15000_15"]))*1.)))))) +
            0.0500000007*np.tanh((((10.0)) * ((((((9.0)) * ((((-1.0*(((((data["f_15000_15"]) < (data["f_0_83"]))*1.))))) - (data["f_0_87"]))))) * 2.0)))) +
            0.0500000007*np.tanh((((((((((((((data["mn"]) < (np.sin(((-1.0*((0.0666666999)))))))*1.)) * 2.0)) * 2.0)) - (2.0))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where((((((data["f_15000_15"]) < (data["f_0_83"]))*1.)) + (data["f_30000_87"]))>0, -3.0, np.where(data["f_0_19"]>0, -3.0, 7.8888897896 ) )) +
            0.0500000007*np.tanh(np.where(np.maximum((((((((data["f_45000_3"]) < (((data["f_0_75"]) / 2.0)))*1.)) + (data["f_75000_23"])))), ((data["f_45000_83"])))<0, 7.8888897896, -3.0 )) +
            0.0500000007*np.tanh(((np.where(((data["f_0_19"]) + ((((data["f_0_15"]) < (np.maximum(((data["f_0_75"])), ((data["f_0_55"])))))*1.)))>0, -3.0, 7.8888897896 )) * 2.0)) +
            0.0500000007*np.tanh((((((((-1.0*(((((data["f_45000_15"]) < (data["f_90000_75"]))*1.))))) - (((((((data["f_0_67"]) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((1.0298500061) - ((((((((data["f_90000_55"]) > (data["f_60000_15"]))*1.)) + ((((data["f_30000_3"]) < (data["f_15000_75"]))*1.)))) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_0_71"]) < (data["f_75000_47"]))*1.)) - ((((((data["mn"]) > (np.sin((data["f_15000_15"]))))*1.)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(np.maximum(((np.maximum((((((data["f_90000_55"]) > (data["f_60000_15"]))*1.))), ((data["f_15000_19"]))))), ((data["f_0_75"])))>0, -3.0, (13.59229755401611328) )) +
            0.0500000007*np.tanh(np.where(((np.minimum(((data["f_0_75"])), ((data["f_0_71"])))) - (np.sin((((-3.0) + (data["f_30000_15"]))))))<0, 9.6666698456, -3.0 )) +
            0.0500000007*np.tanh(((9.6666698456) * (np.cos(((((((data["f_15000_59"]) > ((-1.0*(((((data["f_120000_15"]) < (((data["f_0_55"]) / 2.0)))*1.))))))*1.)) * 2.0)))))) +
            0.0500000007*np.tanh(((((np.cos(((((((data["f_0_87"]) > (data["f_105000_27"]))*1.)) * 2.0)))) - ((((((data["f_0_75"]) > (data["f_30000_7"]))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(((((np.where((((data["f_90000_63"]) > (data["f_0_7"]))*1.)>0, -2.0, ((data["f_0_75"]) * ((-1.0*((9.6666698456))))) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((((((((data["f_0_67"]) * ((-1.0*((9.6666698456)))))) - ((((data["f_0_15"]) < (data["f_75000_75"]))*1.)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_15000_7"]) < ((((data["f_0_83"]) + (data["f_0_87"]))/2.0)))*1.)>0, -3.0, np.where(data["f_60000_71"]>0, -3.0, (10.0) ) )) +
            0.0500000007*np.tanh(((np.where(((((((data["f_0_75"]) + (data["f_0_71"]))/2.0)) > (data["f_15000_15"]))*1.)>0, -2.0, ((2.1351399422) - (data["f_0_67"])) )) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_0_87"]) < (data["f_75000_11"]))*1.)) - ((((((data["f_0_71"]) > (np.minimum(((data["f_15000_15"])), ((data["f_0_79"])))))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh((((((((((-1.0*((((data["f_0_63"]) + ((((((data["f_60000_0"]) * 2.0)) < (data["f_0_71"]))*1.))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((np.cos(((((((data["f_120000_35"]) < (data["f_15000_87"]))*1.)) * 2.0)))) - ((((((data["f_15000_7"]) < (data["f_0_63"]))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(((np.where(np.maximum(((data["f_30000_35"])), ((((data["f_0_75"]) + ((((data["f_30000_71"]) > (data["f_15000_35"]))*1.))))))<0, 9.6666698456, -3.0 )) * 2.0)) +
            0.0500000007*np.tanh((((12.02198886871337891)) * ((((12.02198886871337891)) * ((-1.0*((((data["f_105000_7"]) + ((((data["f_105000_7"]) < (((data["f_0_55"]) / 2.0)))*1.))))))))))) +
            0.0500000007*np.tanh(np.where(np.maximum(((data["f_45000_71"])), ((np.maximum(((data["f_0_67"])), (((((data["f_120000_75"]) > (data["f_105000_15"]))*1.)))))))>0, -3.0, 7.8888897896 )) +
            0.0500000007*np.tanh(np.where((((data["f_0_63"]) > (data["f_45000_15"]))*1.)>0, -3.0, np.where((((data["f_45000_47"]) < (data["f_0_71"]))*1.)>0, -3.0, 9.6666698456 ) )) +
            0.0500000007*np.tanh((((((((-1.0*((((((data["f_0_67"]) * (9.6666698456))) + ((((data["f_75000_15"]) < (data["f_105000_75"]))*1.))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((np.where(((data["f_0_67"]) + ((((data["f_135000_83"]) > (data["f_60000_7"]))*1.)))<0, (((data["f_0_71"]) < (data["f_120000_35"]))*1.), -3.0 )) * 2.0)) +
            0.0500000007*np.tanh((-1.0*((np.where(((((((data["f_75000_11"]) < (data["f_0_87"]))*1.)) + (np.maximum(((data["f_0_83"])), ((data["f_120000_23"])))))/2.0)>0, (10.20698070526123047), -3.0 ))))) +
            0.0500000007*np.tanh(np.where(np.maximum(((data["f_0_63"])), (((((data["f_0_35"]) < (data["f_0_55"]))*1.))))>0, -3.0, (((((data["f_75000_11"]) > (data["f_75000_19"]))*1.)) * 2.0) )) +
            0.0500000007*np.tanh(np.where((((((data["f_15000_79"]) > (((data["f_75000_15"]) / 2.0)))*1.)) + ((((data["f_75000_15"]) < (data["f_0_63"]))*1.)))>0, -3.0, 9.6666698456 )) +
            0.0500000007*np.tanh(((np.where(((data["f_0_63"]) + (np.maximum((((((data["f_30000_55"]) > (data["std"]))*1.))), ((data["f_105000_63"])))))>0, -3.0, 7.8888897896 )) * 2.0)) +
            0.0500000007*np.tanh(np.where(((((((data["f_45000_71"]) > (data["f_30000_35"]))*1.)) + (np.maximum(((data["f_0_75"])), ((data["f_30000_35"])))))/2.0)>0, -3.0, (10.43864536285400391) )) +
            0.0500000007*np.tanh(((np.where(np.maximum(((data["f_135000_31"])), ((np.maximum(((data["f_0_63"])), (((((data["max"]) < (data["mn"]))*1.)))))))>0, -3.0, 9.6666698456 )) * 2.0)) +
            0.0500000007*np.tanh(((np.where(((data["f_0_71"]) + ((((data["f_0_83"]) > (data["f_60000_3"]))*1.)))>0, -3.0, (((data["f_15000_35"]) > (data["f_0_31"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh((-1.0*((((((((((((((data["f_0_71"]) + ((((data["f_135000_83"]) > (data["f_60000_15"]))*1.)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0500000007*np.tanh(np.where((((((((data["f_0_75"]) + (0.1616159976))/2.0)) + (data["f_0_79"]))) + (data["f_0_83"]))>0, -3.0, (9.27371788024902344) )) +
            0.0500000007*np.tanh(np.where(np.maximum(((data["f_135000_39"])), ((((data["f_15000_63"]) + ((((data["f_60000_39"]) < (data["f_75000_71"]))*1.))))))>0, -3.0, 9.6666698456 )) +
            0.0500000007*np.tanh(np.where(np.maximum(((np.maximum(((((data["f_60000_87"]) - (data["f_120000_55"])))), ((data["f_45000_59"]))))), ((data["f_60000_67"])))>0, -3.0, (-1.0*((-3.0))) )) +
            0.0500000007*np.tanh(((np.where(np.maximum(((np.maximum((((((data["f_0_15"]) < (data["f_0_31"]))*1.))), ((data["f_45000_23"]))))), ((data["f_0_67"])))>0, -3.0, (8.0) )) * 2.0)) +
            0.0500000007*np.tanh(((((-2.0) + (((np.where(data["f_0_43"]>0, data["f_135000_19"], 7.8888897896 )) * ((((data["f_75000_39"]) > (data["f_135000_71"]))*1.)))))) * 2.0)) +
            0.0500000007*np.tanh(((((((np.where(data["f_0_63"]<0, np.cos(((((((data["f_60000_0"]) < (data["f_60000_79"]))*1.)) * 2.0))), -3.0 )) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(((data["f_15000_63"]) + ((((data["f_0_83"]) > (data["f_30000_83"]))*1.)))>0, -3.0, np.where(data["f_45000_23"]>0, -3.0, 7.8888897896 ) )) +
            0.0500000007*np.tanh((((((((-1.0*((((((data["f_0_63"]) * 2.0)) + ((((data["f_15000_71"]) > (((data["f_120000_55"]) * 2.0)))*1.))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_45000_23"]) + ((((((data["f_0_83"]) + (((data["f_0_79"]) / 2.0)))) > (data["f_45000_47"]))*1.)))/2.0)>0, -3.0, (8.0) )) +
            0.0500000007*np.tanh(np.where((((data["f_15000_43"]) > ((-1.0*(((((data["f_135000_75"]) > (np.minimum(((data["f_15000_3"])), ((data["f_135000_3"])))))*1.))))))*1.)>0, -3.0, (7.0) )) +
            0.0500000007*np.tanh(((((((((((data["f_0_71"]) + ((((data["std"]) < (data["f_15000_51"]))*1.)))) + (data["f_0_83"]))) * 2.0)) * (-3.0))) * 2.0)) +
            0.0500000007*np.tanh(((np.where(((((((data["f_0_43"]) + ((((data["f_30000_3"]) < (data["f_45000_79"]))*1.)))/2.0)) + (data["f_105000_83"]))/2.0)>0, -3.0, 7.8888897896 )) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_0_75"]>0, -2.0, np.where(data["f_15000_19"]>0, -3.0, (((data["f_45000_23"]) < (data["std"]))*1.) ) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_30000_59"]>0, -3.0, np.where((((data["f_135000_3"]) + ((((data["f_0_79"]) > (data["f_105000_87"]))*1.)))/2.0)>0, -3.0, (11.67853736877441406) ) )) +
            0.0500000007*np.tanh((-1.0*(((((((((((((data["f_15000_15"]) < (data["f_0_11"]))*1.)) + (((((data["f_0_59"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0))))) +
            0.0500000007*np.tanh(np.where(((((((-1.0*((data["min"])))) < (data["f_0_67"]))*1.)) + (data["f_30000_23"]))>0, (-1.0*((9.6666698456))), 9.6666698456 )) +
            0.0500000007*np.tanh(((((np.where(data["f_0_59"]<0, (((data["f_15000_59"]) > (data["f_0_83"]))*1.), -3.0 )) * 2.0)) - ((((data["f_0_71"]) > (data["f_0_47"]))*1.)))) +
            0.0500000007*np.tanh(np.where(np.maximum(((data["f_90000_59"])), ((np.where(data["f_15000_79"]<0, (((data["f_15000_79"]) > (data["f_105000_11"]))*1.), data["f_0_71"] ))))>0, -3.0, (10.04335594177246094) )) +
            0.0500000007*np.tanh(((((np.where(data["f_30000_23"]<0, np.where(data["f_0_71"]<0, (((data["f_105000_63"]) < (data["f_60000_71"]))*1.), -3.0 ), -3.0 )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((np.where(np.maximum(((data["f_0_71"])), (((((data["f_60000_15"]) < (data["f_90000_19"]))*1.))))>0, -3.0, (((data["f_30000_47"]) > (data["f_0_71"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_0_59"]>0, -2.0, (((data["f_90000_11"]) > (np.where(data["f_15000_79"]<0, data["f_0_79"], data["f_0_59"] )))*1.) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_0_43"]) + (data["min"]))/2.0)<0, np.where(data["max"]<0, (13.60011863708496094), (-1.0*(((13.60011863708496094)))) ), (-1.0*(((13.60011863708496094)))) )) +
            0.0500000007*np.tanh(np.where(np.maximum(((data["f_120000_11"])), ((np.maximum(((data["f_0_59"])), (((((data["f_0_15"]) > (data["max"]))*1.)))))))>0, (-1.0*((3.1415927410))), 3.1415927410 )) +
            0.0500000007*np.tanh(((np.where((((((data["f_120000_3"]) > (data["f_120000_0"]))*1.)) + (data["f_0_47"]))>0, -3.0, (((data["f_60000_15"]) > (data["f_30000_7"]))*1.) )) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_90000_3"]) > (data["f_120000_83"]))*1.)) - (data["f_0_71"]))) - ((((data["f_15000_31"]) > (data["f_45000_67"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_105000_71"]) > (np.minimum(((data["f_120000_79"])), ((data["f_15000_87"])))))*1.)) - (0.4777779877))) - (data["f_90000_19"]))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((-1.0*((((((((((data["f_90000_75"]) * 2.0)) + ((((data["f_45000_47"]) < (data["f_0_35"]))*1.)))) * ((8.18460178375244141)))) * ((8.18460178375244141))))))) +
            0.0500000007*np.tanh(((np.where(np.maximum(((data["f_30000_23"])), ((np.maximum(((data["f_135000_79"])), ((((data["f_45000_51"]) - (data["max"]))))))))>0, -3.0, 3.0 )) * 2.0)) +
            0.0500000007*np.tanh(((((9.6666698456) * (((9.6666698456) * ((((((data["f_0_83"]) < (data["f_75000_3"]))*1.)) - (data["f_0_79"]))))))) - (9.6666698456))) +
            0.0500000007*np.tanh((((((((((data["f_15000_83"]) < (data["f_30000_75"]))*1.)) - ((((data["f_0_71"]) + ((((data["f_45000_79"]) > (data["f_45000_3"]))*1.)))/2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((np.where(data["f_15000_59"]>0, -3.0, (((((data["f_75000_71"]) < (data["f_45000_31"]))*1.)) + ((((data["f_15000_0"]) > (data["f_105000_83"]))*1.))) )) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_15000_79"]) < (data["f_105000_87"]))*1.)) - (data["f_15000_11"]))) - ((((((data["f_15000_71"]) / 2.0)) > (data["f_120000_55"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh(((((np.where((((data["max"]) < (data["f_60000_19"]))*1.)>0, -3.0, (((data["f_75000_15"]) > (data["f_0_63"]))*1.) )) * 2.0)) - (0.4777779877))) +
            0.0500000007*np.tanh((((((((((data["f_15000_31"]) < (np.minimum(((data["f_30000_31"])), ((data["f_45000_67"])))))*1.)) * 2.0)) - ((((data["f_105000_71"]) < (data["f_15000_87"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh((((((((((((np.minimum(((np.sin((data["f_45000_27"])))), ((data["f_0_3"])))) > (data["f_90000_67"]))*1.)) * 2.0)) - (1.0))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_60000_71"]) > (data["f_105000_43"]))*1.)) + ((((((data["f_0_79"]) < (data["f_75000_7"]))*1.)) - (0.4777779877))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_135000_59"]) < (data["f_120000_39"]))*1.)) - (data["f_90000_19"]))) - ((((data["f_135000_59"]) > (data["f_120000_39"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_90000_59"]) > (data["f_105000_43"]))*1.)) + (((np.where(data["f_0_31"]>0, -3.0, (((data["f_90000_63"]) < (data["f_15000_35"]))*1.) )) * 2.0)))) +
            0.0500000007*np.tanh(np.where(data["f_15000_19"]>0, -3.0, (((((((data["f_15000_47"]) / 2.0)) > (np.where(data["f_120000_83"]<0, data["f_90000_87"], data["f_15000_83"] )))*1.)) * 2.0) )) +
            0.0500000007*np.tanh((((((((((data["f_135000_31"]) > (((data["f_120000_71"]) / 2.0)))*1.)) * 2.0)) - ((((((data["f_15000_83"]) / 2.0)) > (data["f_120000_71"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh((((((np.tanh((data["f_105000_15"]))) > (np.where(data["f_60000_79"]<0, np.maximum(((data["f_0_79"])), ((data["f_90000_11"]))), data["f_0_71"] )))*1.)) - (data["f_0_71"]))) +
            0.0500000007*np.tanh((-1.0*((((data["max"]) + (np.where((((data["f_15000_35"]) > (data["f_90000_63"]))*1.)>0, -3.0, (((data["f_30000_71"]) < (data["f_90000_63"]))*1.) ))))))) +
            0.0500000007*np.tanh(((((((((np.where(data["f_60000_83"]>0, data["f_135000_31"], (((data["max"]) > (data["f_0_11"]))*1.) )) * 2.0)) + (-1.0))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_120000_47"]) > (data["f_120000_71"]))*1.)) - (np.where(data["f_120000_79"]<0, (((data["f_105000_39"]) < (data["f_120000_71"]))*1.), data["f_30000_43"] )))) * 2.0)) +
            0.0500000007*np.tanh((((((((((np.maximum(((data["f_15000_47"])), ((data["f_60000_43"])))) > (((data["f_30000_79"]) + (data["f_120000_23"]))))*1.)) * 2.0)) - (0.9200000167))) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_0_83"]<0, (((((data["f_105000_71"]) * 2.0)) > (np.where(data["f_45000_47"]>0, data["f_105000_23"], data["f_0_67"] )))*1.), (-1.0*(((10.0)))) )) +
            0.0500000007*np.tanh((((((((((((data["f_30000_47"]) > (data["f_30000_67"]))*1.)) * 2.0)) - (((((-1.0*((data["min"])))) < (data["f_30000_43"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_120000_19"]) > (data["f_120000_63"]))*1.)) * 2.0)) - ((((data["f_45000_79"]) > (data["f_60000_79"]))*1.)))) - (data["f_0_43"]))) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_0_11"]) < (data["f_0_7"]))*1.)) - (np.where(np.where(data["f_0_87"]>0, data["f_0_79"], data["f_0_31"] )>0, 7.8888897896, data["f_0_87"] )))) +
            0.0500000007*np.tanh((((((((((data["f_75000_23"]) < (data["f_45000_51"]))*1.)) - (((((((data["f_15000_71"]) < (data["f_45000_51"]))*1.)) + (data["f_0_39"]))/2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_120000_63"]) < (data["f_105000_87"]))*1.)) - ((((((np.maximum(((data["f_120000_55"])), ((data["f_90000_63"])))) < (data["f_45000_87"]))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_90000_71"]) < (data["f_75000_55"]))*1.)>0, ((7.8888897896) - (data["f_15000_23"])), (-1.0*(((((data["f_90000_71"]) < (data["f_105000_59"]))*1.)))) )) +
            0.0500000007*np.tanh((-1.0*((((np.where(data["f_30000_39"]>0, data["f_0_0"], np.where(data["f_90000_67"]<0, (((data["f_45000_43"]) > (data["f_45000_3"]))*1.), -3.0 ) )) * 2.0))))) +
            0.0500000007*np.tanh(((9.6666698456) * ((-1.0*(((((((((data["f_45000_71"]) < (data["f_105000_19"]))*1.)) / 2.0)) - ((((data["f_105000_63"]) < (data["f_105000_19"]))*1.))))))))) +
            0.0500000007*np.tanh((((((data["f_15000_15"]) > (data["f_0_11"]))*1.)) - (((((((data["f_135000_79"]) + ((((data["f_15000_31"]) > (data["std"]))*1.)))) * 2.0)) * 2.0)))) +
            0.0500000007*np.tanh((((((data["f_45000_51"]) < (np.sin((data["max"]))))*1.)) - (np.where(data["f_105000_67"]>0, data["f_15000_11"], (((data["f_75000_19"]) > (data["f_45000_51"]))*1.) )))) +
            0.0500000007*np.tanh(((((np.where(data["f_15000_63"]>0, -3.0, (((data["f_90000_43"]) < (data["f_15000_51"]))*1.) )) * 2.0)) - ((((data["f_120000_31"]) > (data["f_45000_0"]))*1.)))) +
            0.0500000007*np.tanh(((((np.where(data["f_45000_87"]>0, -2.0, np.where(data["f_30000_59"]>0, -2.0, (((data["f_30000_75"]) > (data["f_30000_15"]))*1.) ) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_30000_3"]) > (data["f_120000_51"]))*1.)) - (data["f_0_11"]))) * 2.0)) - ((((data["f_45000_67"]) < (data["f_15000_31"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_75000_43"]) < (data["std"]))*1.)) * 2.0)) - ((((data["max"]) < (data["f_75000_67"]))*1.)))) - (data["std"]))) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_30000_11"]>0, -2.0, (((data["f_60000_71"]) > (np.where(data["f_105000_67"]>0, -2.0, data["f_105000_63"] )))*1.) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((np.where(((data["f_0_31"]) - (data["f_15000_35"]))>0, -3.0, (((((data["f_60000_71"]) > (data["f_105000_63"]))*1.)) - (data["f_30000_87"])) )) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_120000_55"]) > (np.where(np.maximum(((data["f_0_79"])), ((data["f_45000_83"])))>0, data["f_135000_59"], data["f_0_79"] )))*1.)) - (0.3636359870))) * 2.0)) +
            0.0500000007*np.tanh(((((((((((((((data["f_105000_67"]) < (data["f_75000_39"]))*1.)) * (data["f_0_35"]))) < (data["f_105000_67"]))*1.)) * 2.0)) * 2.0)) - (1.6046500206))) +
            0.0500000007*np.tanh(((((np.where(data["f_135000_27"]>0, -3.0, (((((np.tanh((data["f_120000_19"]))) < (data["f_15000_3"]))*1.)) * 2.0) )) * 2.0)) - (data["f_0_35"]))) +
            0.0500000007*np.tanh((((((((((((((data["f_60000_55"]) > (data["f_45000_59"]))*1.)) * 2.0)) - (data["f_15000_7"]))) - (np.cos((data["f_60000_83"]))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_135000_63"]>0, np.where(data["f_60000_63"]>0, (-1.0*(((14.95171737670898438)))), (14.95172023773193359) ), np.where(data["f_120000_83"]<0, -3.0, (14.95172023773193359) ) )) +
            0.0500000007*np.tanh(((((np.where(data["f_90000_39"]>0, -2.0, (((np.minimum(((data["f_90000_63"])), ((data["f_75000_3"])))) < (np.sin((data["f_15000_51"]))))*1.) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_30000_0"]) > ((((data["f_30000_23"]) + ((-1.0*(((((data["f_30000_55"]) > (data["f_90000_63"]))*1.))))))/2.0)))*1.)) * 2.0)) - (0.6739130020))) +
            0.0500000007*np.tanh((((((((((data["f_135000_35"]) > (((data["f_135000_59"]) / 2.0)))*1.)) - ((((data["f_135000_75"]) < (data["f_0_0"]))*1.)))) * 2.0)) - (data["f_15000_3"]))) +
            0.0500000007*np.tanh(((((np.where(np.where(data["f_90000_79"]<0, data["f_135000_79"], data["f_0_87"] )<0, (((data["f_45000_43"]) < (data["f_15000_3"]))*1.), -3.0 )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((((data["f_75000_39"]) > (data["f_90000_55"]))*1.)) * 2.0)) - ((((data["f_90000_55"]) < (data["f_105000_63"]))*1.)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((-1.0*((np.where((((data["f_105000_71"]) > (((data["f_0_63"]) / 2.0)))*1.)>0, -3.0, (((data["f_0_0"]) > (data["f_45000_7"]))*1.) ))))) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_105000_11"]) < (data["f_60000_0"]))*1.)) + (data["f_135000_83"]))) - ((((data["f_60000_15"]) < (data["f_90000_19"]))*1.)))) - (data["f_15000_7"]))) +
            0.0500000007*np.tanh((((((((((data["f_75000_67"]) < (np.where(data["f_120000_79"]<0, data["f_0_51"], ((data["f_120000_83"]) - (data["f_0_51"])) )))*1.)) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_15000_47"]) > (data["f_30000_67"]))*1.)) - ((((data["f_105000_87"]) < ((((data["f_30000_87"]) + (((data["f_30000_87"]) / 2.0)))/2.0)))*1.)))) +
            0.0500000007*np.tanh(((np.where(data["f_90000_7"]<0, (((data["f_90000_71"]) > ((((data["f_90000_47"]) + (np.minimum(((data["f_90000_7"])), ((data["f_0_87"])))))/2.0)))*1.), -3.0 )) * 2.0)) +
            0.0499804616*np.tanh(((((np.where(data["f_90000_67"]<0, ((data["min"]) - ((((data["f_120000_83"]) < (data["f_30000_87"]))*1.))), 1.6046500206 )) * 2.0)) - (data["f_0_0"]))) +
            0.0500000007*np.tanh(np.cos((((((np.tanh((np.maximum((((((data["f_0_31"]) > (data["f_45000_67"]))*1.))), (((((data["f_15000_11"]) > (data["f_30000_15"]))*1.))))))) * 2.0)) * 2.0)))) +
            0.0500000007*np.tanh((((((((((data["f_0_83"]) < (((data["f_30000_83"]) + (data["f_75000_79"]))))*1.)) * 2.0)) - ((((data["f_0_27"]) < (data["f_75000_67"]))*1.)))) * 2.0)) +
            0.0499804616*np.tanh((((data["f_15000_47"]) > (np.where(data["f_135000_75"]>0, ((data["f_75000_47"]) * 2.0), np.minimum(((((data["f_90000_87"]) * 2.0))), ((((data["f_15000_79"]) * 2.0)))) )))*1.)) +
            0.0500000007*np.tanh(np.where(data["f_45000_67"]<0, (((data["f_120000_75"]) > (np.where(data["f_135000_75"]<0, data["f_15000_79"], ((data["f_45000_75"]) - (data["f_15000_79"])) )))*1.), -3.0 )) +
            0.0500000007*np.tanh((((((data["f_15000_11"]) < (np.where(data["f_60000_87"]<0, data["f_30000_51"], ((data["f_30000_51"]) * 2.0) )))*1.)) - ((((data["f_75000_67"]) > (data["f_30000_51"]))*1.)))) +
            0.0500000007*np.tanh((((((((((data["f_60000_63"]) < (data["f_0_59"]))*1.)) - ((((((data["f_0_59"]) > (data["f_15000_59"]))*1.)) + (data["f_30000_79"]))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_75000_71"]) > (data["f_105000_47"]))*1.)) * 2.0)) - (((data["f_15000_0"]) + ((((data["f_105000_87"]) < (data["f_0_79"]))*1.)))))) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_60000_79"]>0, (-1.0*(((((data["f_15000_47"]) > (data["f_75000_31"]))*1.)))), (((data["f_0_47"]) > (np.minimum(((data["f_45000_63"])), ((data["f_15000_67"])))))*1.) )) +
            0.0500000007*np.tanh((((((data["f_45000_3"]) > (data["f_45000_43"]))*1.)) - (((np.where(data["f_105000_23"]>0, data["f_0_7"], (((data["f_105000_23"]) > (data["f_45000_3"]))*1.) )) * 2.0)))) +
            0.0500000007*np.tanh(np.where(data["f_15000_79"]<0, (-1.0*(((((data["f_60000_23"]) > (data["f_60000_47"]))*1.)))), (((((data["f_60000_11"]) > (data["f_60000_47"]))*1.)) - (data["f_60000_11"])) )) +
            0.0500000007*np.tanh((((((((data["f_15000_3"]) < (data["f_60000_71"]))*1.)) + (((((data["f_15000_55"]) - ((((data["f_75000_47"]) > (data["f_45000_3"]))*1.)))) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh((((((((np.where(data["f_105000_87"]<0, data["f_105000_71"], data["f_30000_67"] )) < (np.maximum(((data["f_30000_67"])), ((np.sin((data["f_75000_39"])))))))*1.)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_120000_75"]) > (data["f_135000_11"]))*1.)) - ((((data["f_60000_47"]) < (np.where(data["f_135000_3"]<0, data["f_45000_63"], (-1.0*((data["f_90000_23"]))) )))*1.)))) +
            0.0500000007*np.tanh(((((np.sin(((((((((((((data["f_15000_31"]) < (data["f_30000_19"]))*1.)) + (data["f_60000_27"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((data["f_15000_47"]) > (np.where(data["f_0_47"]<0, np.minimum(((data["f_45000_67"])), ((data["f_15000_67"]))), (((data["f_45000_67"]) < (data["f_45000_43"]))*1.) )))*1.)) +
            0.0500000007*np.tanh(((np.where(data["f_90000_51"]>0, 3.1415927410, ((0.4777779877) - ((((((data["f_15000_3"]) < (data["f_120000_19"]))*1.)) * 2.0))) )) - (data["f_15000_3"]))) +
            0.0500000007*np.tanh(((((((((data["f_60000_63"]) - (data["f_0_15"]))) - (((data["f_0_11"]) + ((((data["f_0_83"]) > (data["f_75000_3"]))*1.)))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_105000_43"]) < (data["f_90000_59"]))*1.)) * 2.0)) - ((((data["f_90000_43"]) < (data["f_90000_63"]))*1.)))) * 2.0)) - (data["f_0_7"]))) +
            0.0499804616*np.tanh(np.where((((data["f_15000_15"]) < (data["f_0_11"]))*1.)>0, (-1.0*(((11.68953132629394531)))), (((((data["f_30000_7"]) < (data["f_90000_39"]))*1.)) - (data["f_60000_3"])) )) +
            0.0500000007*np.tanh((((((((data["f_60000_63"]) < (np.where(data["f_135000_75"]>0, data["f_120000_43"], data["f_0_59"] )))*1.)) * 2.0)) - ((((data["f_120000_7"]) > (data["f_60000_83"]))*1.)))) +
            0.0500000007*np.tanh(((((np.where(data["f_90000_59"]<0, (((data["f_135000_59"]) < (data["f_120000_39"]))*1.), data["f_135000_83"] )) - ((((data["f_45000_0"]) > (data["f_90000_27"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh(((((data["f_0_27"]) + ((((((data["f_120000_47"]) > (((data["f_15000_71"]) / 2.0)))*1.)) + ((((data["f_90000_43"]) < (data["mn"]))*1.)))))) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_60000_63"]) < (data["f_15000_47"]))*1.)) - (((np.maximum(((data["f_90000_71"])), (((((data["f_15000_59"]) > (data["f_90000_67"]))*1.))))) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(((np.where(data["f_135000_3"]>0, -3.0, (((data["f_15000_47"]) > (((data["f_90000_87"]) * 2.0)))*1.) )) - ((((data["f_0_55"]) > (data["f_30000_3"]))*1.)))) +
            0.0500000007*np.tanh(np.where(data["f_105000_19"]>0, np.cos((data["f_90000_35"])), (((((data["f_105000_19"]) < (data["f_90000_35"]))*1.)) - ((((data["f_90000_35"]) > (data["f_60000_7"]))*1.))) )) +
            0.0500000007*np.tanh((((((((data["f_60000_0"]) < (data["f_105000_51"]))*1.)) - ((((data["f_90000_51"]) > (data["f_120000_31"]))*1.)))) - ((((data["f_105000_51"]) > (data["f_120000_59"]))*1.)))) +
            0.0500000007*np.tanh((((((((data["f_135000_43"]) < (data["f_75000_3"]))*1.)) - ((((data["f_105000_35"]) < (np.where(data["f_30000_47"]<0, data["f_60000_47"], data["f_90000_7"] )))*1.)))) * 2.0)) +
            0.0500000007*np.tanh(((((np.maximum((((((((data["f_120000_75"]) + (data["f_120000_87"]))) > (data["f_90000_67"]))*1.))), (((((data["f_30000_71"]) > (data["f_90000_71"]))*1.))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_15000_19"]) > (data["f_15000_55"]))*1.)) * 2.0)) - ((((((((data["f_15000_55"]) > (((data["f_120000_55"]) * 2.0)))*1.)) * 2.0)) * 2.0)))) +
            0.0500000007*np.tanh(np.where(np.where(data["f_0_83"]>0, data["f_105000_67"], (((data["f_120000_63"]) < (data["f_90000_27"]))*1.) )>0, (((10.0)) - (data["f_0_11"])), -2.0 )) +
            0.0499609224*np.tanh(((((((np.cos(((((((data["f_45000_15"]) < (np.minimum(((data["f_135000_87"])), ((((data["f_120000_19"]) * 2.0))))))*1.)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_45000_75"]<0, (-1.0*(((((data["f_135000_83"]) < (((data["f_60000_75"]) * ((((data["f_0_0"]) > (data["f_90000_23"]))*1.)))))*1.)))), 1.0816299915 )) +
            0.0500000007*np.tanh((-1.0*((np.where((((data["f_15000_31"]) > (data["f_45000_67"]))*1.)>0, 7.8888897896, ((data["f_135000_67"]) - ((((data["f_105000_15"]) > (data["f_45000_67"]))*1.))) ))))) +
            0.0500000007*np.tanh((((((((data["f_0_59"]) > (np.where(data["f_75000_79"]>0, data["f_0_63"], np.where(data["f_75000_83"]>0, data["f_60000_47"], data["f_60000_63"] ) )))*1.)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_45000_15"]) > (data["f_90000_51"]))*1.)) - ((((np.maximum(((data["f_120000_23"])), ((data["f_15000_75"])))) > (data["f_120000_51"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_75000_3"]<0, (((data["f_15000_87"]) < (((data["f_90000_87"]) + (np.sin((data["f_120000_3"]))))))*1.), -3.0 )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(data["f_120000_83"]<0, (-1.0*(((((data["f_15000_27"]) > (data["f_60000_35"]))*1.)))), (((data["f_60000_39"]) < (((data["f_0_79"]) + (data["f_75000_35"]))))*1.) )) +
            0.0500000007*np.tanh(((((np.where(data["f_0_47"]>0, data["f_120000_47"], (((data["f_30000_67"]) < (np.where(data["f_60000_79"]<0, data["f_0_47"], data["f_30000_11"] )))*1.) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_105000_43"]) > (data["f_135000_59"]))*1.)) - (np.maximum((((((data["f_60000_71"]) < (data["f_0_11"]))*1.))), (((((data["f_0_63"]) > (data["f_30000_35"]))*1.))))))) +
            0.0499804616*np.tanh((((np.where(data["f_135000_59"]<0, data["f_75000_63"], data["f_120000_71"] )) > (np.where(data["f_120000_71"]<0, data["f_105000_59"], (((data["f_45000_79"]) < (data["f_75000_63"]))*1.) )))*1.)) +
            0.0500000007*np.tanh(np.where((((data["f_120000_3"]) < (data["f_105000_55"]))*1.)>0, -3.0, np.where(data["f_135000_27"]>0, -3.0, (((data["f_90000_0"]) < (data["f_30000_67"]))*1.) ) )) +
            0.0500000007*np.tanh((((((((((data["f_105000_79"]) < (np.sin((data["f_75000_0"]))))*1.)) - ((((data["f_90000_55"]) < (np.sin((data["f_15000_83"]))))*1.)))) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh(((((((((data["f_30000_43"]) + ((((data["f_30000_43"]) < (((np.maximum(((data["f_90000_71"])), ((data["f_60000_67"])))) * 2.0)))*1.)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((((data["f_90000_55"]) < (np.minimum(((data["f_75000_39"])), ((data["f_90000_3"])))))*1.)) - (0.1707320064))) - (data["f_45000_43"]))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_60000_59"]>0, data["f_135000_3"], np.where(data["f_135000_3"]<0, (((data["f_75000_47"]) < (data["f_0_3"]))*1.), -2.0 ) )) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh(np.where(((data["f_105000_47"]) + ((((data["f_105000_47"]) > (data["f_120000_75"]))*1.)))>0, -2.0, (((np.tanh((data["f_75000_55"]))) > (data["f_105000_43"]))*1.) )) +
            0.0500000007*np.tanh((((data["f_30000_47"]) > (np.where(data["f_90000_55"]>0, data["f_45000_11"], np.where(data["f_90000_71"]>0, data["f_30000_47"], np.minimum(((data["f_75000_63"])), ((data["f_135000_75"]))) ) )))*1.)) +
            0.0500000007*np.tanh((((((((data["f_90000_43"]) > (data["f_60000_51"]))*1.)) - (np.where(data["f_60000_51"]>0, data["f_90000_19"], (((data["f_60000_19"]) < (data["f_30000_47"]))*1.) )))) * 2.0)) +
            0.0499804616*np.tanh(((((np.where(data["f_15000_79"]<0, (-1.0*(((((data["f_45000_55"]) < (data["f_45000_35"]))*1.)))), (((data["f_135000_63"]) > (data["f_105000_51"]))*1.) )) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh(np.where(data["f_30000_87"]>0, data["min"], (((((data["f_30000_55"]) < (((data["f_45000_0"]) * 2.0)))*1.)) + ((((data["std"]) > (data["f_75000_23"]))*1.))) )) +
            0.0500000007*np.tanh(((((((((0.3636359870) - ((((data["f_45000_63"]) > (np.where(data["f_105000_83"]<0, data["f_0_75"], data["f_30000_47"] )))*1.)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.maximum((((((((data["f_135000_71"]) < (np.where(data["f_135000_83"]<0, data["f_45000_35"], data["f_105000_19"] )))*1.)) * 2.0))), (((((data["f_30000_35"]) > (data["f_75000_39"]))*1.))))) +
            0.0500000007*np.tanh(np.where((((data["f_75000_71"]) > (data["f_90000_55"]))*1.)>0, -3.0, np.cos(((((((((data["f_0_87"]) > (data["f_75000_83"]))*1.)) * 2.0)) * 2.0))) )) +
            0.0500000007*np.tanh(np.where(data["f_30000_47"]<0, (((((((data["f_45000_15"]) > (data["f_60000_35"]))*1.)) - ((((data["f_90000_19"]) > (data["f_60000_15"]))*1.)))) * 2.0), data["f_120000_47"] )) +
            0.0499609224*np.tanh(np.where((((data["f_75000_55"]) > (np.where(data["f_120000_79"]<0, data["f_105000_55"], data["f_60000_43"] )))*1.)>0, (8.0), ((data["f_105000_55"]) + (-1.0)) )) +
            0.0500000007*np.tanh(np.where(data["f_90000_3"]>0, -3.0, (((((((data["f_75000_3"]) < (data["max"]))*1.)) + ((((data["f_120000_39"]) > (data["f_135000_59"]))*1.)))) * 2.0) )) +
            0.0499804616*np.tanh((((((((((data["f_45000_63"]) < (data["f_135000_83"]))*1.)) - ((((((data["f_0_43"]) > (data["f_0_67"]))*1.)) + (data["f_135000_83"]))))) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh((((((data["f_60000_23"]) < (data["f_75000_59"]))*1.)) - ((((((data["f_45000_87"]) > (data["f_120000_83"]))*1.)) + ((((data["f_60000_23"]) > (data["f_75000_59"]))*1.)))))) +
            0.0499804616*np.tanh((((((((((data["f_135000_67"]) < (data["f_105000_43"]))*1.)) - (((0.4777779877) - ((((data["f_30000_23"]) > (data["f_60000_75"]))*1.)))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_135000_63"]) > (data["f_90000_23"]))*1.)) - ((((data["f_90000_87"]) > (data["f_120000_87"]))*1.)))) - ((((data["f_0_83"]) > (data["f_120000_87"]))*1.)))) +
            0.0500000007*np.tanh((((((((((((data["f_90000_79"]) > (np.where(data["f_135000_75"]<0, data["f_15000_79"], data["f_90000_63"] )))*1.)) - (0.3183098733))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((np.where(data["f_135000_87"]<0, (((data["f_90000_63"]) > (((data["f_105000_51"]) / 2.0)))*1.), (((data["f_30000_15"]) > (data["f_15000_39"]))*1.) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_45000_0"]) < (((data["f_0_15"]) - (data["f_120000_3"]))))*1.)) - ((((((data["f_45000_63"]) > (data["f_120000_43"]))*1.)) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(((np.where(data["f_120000_47"]>0, -3.0, (((((data["f_135000_63"]) < (data["f_30000_51"]))*1.)) + ((((data["f_120000_7"]) < (data["f_75000_3"]))*1.))) )) * 2.0)) +
            0.0499804616*np.tanh((((((((((data["f_0_43"]) + ((((data["f_75000_71"]) > (data["f_105000_43"]))*1.)))/2.0)) - ((((data["f_45000_19"]) < (data["f_15000_35"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(((data["f_135000_75"]) + (data["f_45000_27"]))>0, (((data["f_135000_83"]) > (data["f_90000_15"]))*1.), (-1.0*(((((data["f_135000_83"]) > (data["f_120000_47"]))*1.)))) )) +
            0.0499804616*np.tanh((((((data["f_90000_23"]) < (np.minimum(((data["f_135000_63"])), ((np.where((((data["f_90000_23"]) > (data["f_75000_7"]))*1.)>0, 0.3183098733, data["f_0_0"] ))))))*1.)) * 2.0)) +
            0.0500000007*np.tanh(((((((np.where((((data["f_90000_83"]) < (data["f_75000_87"]))*1.)>0, -2.0, (((data["f_15000_83"]) < (data["f_120000_3"]))*1.) )) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(((((((data["f_15000_35"]) > (data["f_75000_19"]))*1.)) > ((((0.1616159976) + (((data["f_45000_55"]) + (((data["f_45000_83"]) + (data["f_135000_59"]))))))/2.0)))*1.)) +
            0.0500000007*np.tanh(((((-3.0) * (((data["f_15000_11"]) + ((((data["f_15000_35"]) < (np.where(data["f_120000_75"]<0, data["f_0_31"], data["f_75000_27"] )))*1.)))))) * 2.0)) +
            0.0499804616*np.tanh((((((((data["f_105000_19"]) < (np.minimum(((data["f_30000_7"])), (((-1.0*((data["f_15000_31"]))))))))*1.)) - ((((data["f_15000_31"]) > (data["f_45000_67"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh(((((data["f_15000_55"]) + (np.maximum((((((data["f_75000_39"]) < (data["f_30000_35"]))*1.))), (((((data["f_45000_23"]) < (((data["f_45000_0"]) * 2.0)))*1.))))))) * 2.0)) +
            0.0499804616*np.tanh((-1.0*((((np.sin((((((((data["f_0_71"]) * 2.0)) + ((((data["f_45000_71"]) < (((data["f_120000_39"]) * 2.0)))*1.)))) * 2.0)))) * 2.0))))) +
            0.0499804616*np.tanh((((((data["f_45000_63"]) < (data["f_0_19"]))*1.)) + ((((((data["f_30000_55"]) < (data["f_0_19"]))*1.)) - ((((data["f_15000_3"]) < (data["f_105000_35"]))*1.)))))) +
            0.0500000007*np.tanh((((((((((data["f_90000_55"]) > (((data["f_60000_47"]) - (data["f_90000_3"]))))*1.)) * 2.0)) - ((((data["f_60000_47"]) < (data["f_45000_35"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh(((((np.where((((data["f_120000_15"]) < (data["f_120000_23"]))*1.)>0, -3.0, (((data["f_45000_35"]) > (((data["f_135000_43"]) * 2.0)))*1.) )) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh((((data["f_135000_67"]) > (np.where(data["f_30000_67"]<0, data["f_135000_59"], np.where(((data["f_135000_59"]) - (data["f_135000_67"]))>0, data["f_30000_11"], (7.0) ) )))*1.)) +
            0.0500000007*np.tanh(((((((((data["f_75000_71"]) > (data["f_30000_15"]))*1.)) > ((((data["f_105000_63"]) > (data["f_105000_19"]))*1.)))*1.)) - ((((data["f_105000_63"]) > (data["f_105000_19"]))*1.)))) +
            0.0500000007*np.tanh(((np.where(data["f_45000_79"]>0, data["f_60000_87"], (((data["f_135000_75"]) > ((((data["f_30000_27"]) + (data["f_60000_79"]))/2.0)))*1.) )) - (((1.0816299915) / 2.0)))) +
            0.0500000007*np.tanh((((data["f_30000_0"]) > (((np.where(data["f_45000_43"]<0, np.where(data["f_135000_59"]<0, data["f_15000_67"], data["f_30000_19"] ), data["f_30000_0"] )) - (data["f_90000_71"]))))*1.)) +
            0.0500000007*np.tanh(((((((((data["f_30000_63"]) + (((data["f_60000_87"]) + ((((((data["f_30000_59"]) / 2.0)) < (data["f_120000_35"]))*1.)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_0_31"]) < (np.minimum(((data["f_120000_63"])), ((data["f_45000_67"])))))*1.)) - ((((data["f_45000_67"]) > (np.minimum(((data["f_90000_35"])), ((data["f_60000_47"])))))*1.)))) +
            0.0500000007*np.tanh(np.where(np.sin((((data["f_45000_23"]) - (data["f_105000_63"]))))<0, (((data["f_15000_47"]) > (((data["f_105000_63"]) + (data["f_45000_79"]))))*1.), -1.0 )) +
            0.0500000007*np.tanh((((((data["f_90000_59"]) > (data["f_105000_43"]))*1.)) - ((((data["f_45000_71"]) > (np.where(data["f_135000_87"]>0, np.sin((data["f_75000_79"])), data["f_45000_63"] )))*1.)))) +
            0.0500000007*np.tanh((((((((data["f_75000_27"]) > (data["f_45000_7"]))*1.)) - ((((data["f_135000_75"]) < (data["f_0_0"]))*1.)))) - ((((data["f_15000_11"]) > (data["f_45000_7"]))*1.)))) +
            0.0499804616*np.tanh((((((data["f_75000_7"]) > (data["f_105000_31"]))*1.)) - ((((((data["f_105000_51"]) - (((data["f_120000_67"]) * (data["f_75000_75"]))))) > (data["f_120000_3"]))*1.)))) +
            0.0500000007*np.tanh((((((((data["f_120000_67"]) < (data["f_60000_71"]))*1.)) - ((((data["f_0_43"]) > (data["f_45000_71"]))*1.)))) - ((((data["f_90000_23"]) > (data["f_105000_59"]))*1.)))) +
            0.0499804616*np.tanh((((((((data["f_30000_67"]) > (((data["f_90000_0"]) - ((((data["f_30000_3"]) > (((data["f_15000_63"]) - (data["f_105000_71"]))))*1.)))))*1.)) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((data["f_90000_35"]) < (data["f_135000_43"]))*1.)) - ((((((data["f_135000_47"]) < (np.where(data["f_135000_51"]<0, data["f_135000_51"], data["f_135000_43"] )))*1.)) * 2.0)))) +
            0.0500000007*np.tanh((((((data["f_30000_47"]) < (data["f_0_23"]))*1.)) - (np.where(data["f_75000_87"]>0, ((data["f_135000_67"]) * 2.0), (((data["f_0_59"]) < (data["f_30000_43"]))*1.) )))) +
            0.0500000007*np.tanh(np.where(data["f_0_87"]<0, data["f_135000_47"], np.cos((np.where(data["f_60000_51"]>0, data["f_0_35"], np.where(data["f_90000_51"]>0, data["f_135000_3"], (9.30300045013427734) ) ))) )) +
            0.0499804616*np.tanh((((((((data["f_105000_43"]) - (data["f_0_39"]))) > ((((data["f_45000_71"]) < (data["f_90000_71"]))*1.)))*1.)) - ((((data["f_75000_43"]) < (data["f_90000_71"]))*1.)))) +
            0.0500000007*np.tanh(((((np.where(data["f_135000_55"]>0, data["f_90000_75"], np.where(data["f_135000_67"]>0, data["f_135000_63"], (((data["f_45000_27"]) > (data["f_90000_63"]))*1.) ) )) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh(np.where(np.where(data["min"]<0, data["f_75000_83"], (((data["f_60000_0"]) > (data["f_105000_83"]))*1.) )>0, -3.0, (((data["f_120000_43"]) > (data["f_135000_83"]))*1.) )) +
            0.0500000007*np.tanh(np.where(data["f_75000_75"]>0, data["f_60000_55"], (((((np.tanh((data["f_15000_47"]))) > (data["f_30000_67"]))*1.)) - ((((data["f_30000_39"]) > (data["f_45000_27"]))*1.))) )) +
            0.0499804616*np.tanh((((((((((data["f_75000_27"]) > (data["f_135000_67"]))*1.)) * 2.0)) - ((((((data["f_0_87"]) > (data["f_135000_67"]))*1.)) + (data["f_135000_67"]))))) * 2.0)) +
            0.0499609224*np.tanh((((((((((data["f_45000_71"]) > (data["f_0_83"]))*1.)) * 2.0)) - ((((((data["f_120000_71"]) > (data["f_0_83"]))*1.)) + (data["f_45000_3"]))))) * 2.0)) +
            0.0499609224*np.tanh((((((((((data["f_120000_51"]) > (data["f_120000_55"]))*1.)) + (((data["f_15000_83"]) - ((((data["f_105000_27"]) > (data["f_30000_3"]))*1.)))))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((((data["f_120000_87"]) > (((((data["f_60000_47"]) / 2.0)) / 2.0)))*1.)) * 2.0)) - ((((data["f_45000_31"]) > (data["f_60000_47"]))*1.)))) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_75000_47"]) < (data["f_120000_31"]))*1.)) - ((((data["f_75000_47"]) > (data["f_30000_83"]))*1.)))) - ((((data["f_60000_11"]) > (data["f_60000_7"]))*1.)))) +
            0.0500000007*np.tanh((((((((((data["f_105000_43"]) > (data["f_135000_39"]))*1.)) * 2.0)) - (((data["f_0_3"]) + ((((data["f_45000_67"]) > (data["f_105000_19"]))*1.)))))) * 2.0)) +
            0.0499804616*np.tanh(np.where((((data["f_135000_15"]) < (data["f_120000_71"]))*1.)>0, -2.0, (((data["f_45000_31"]) > (np.where(data["f_45000_31"]<0, data["f_15000_11"], 0.1898729950 )))*1.) )) +
            0.0499413870*np.tanh(((((((((data["f_75000_55"]) + (np.where(data["f_75000_51"]>0, data["f_60000_87"], (((data["f_30000_31"]) > (data["f_75000_71"]))*1.) )))) * 2.0)) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh(np.where(data["f_60000_75"]>0, data["f_135000_0"], (((data["f_105000_71"]) > ((((data["f_90000_75"]) + (np.where(data["f_135000_67"]<0, data["f_30000_87"], data["f_135000_0"] )))/2.0)))*1.) )) +
            0.0500000007*np.tanh(((np.where(data["f_45000_47"]>0, data["f_105000_83"], (((((data["f_15000_7"]) > (data["f_75000_31"]))*1.)) - ((((data["f_15000_7"]) < (data["f_45000_47"]))*1.))) )) * 2.0)) +
            0.0500000007*np.tanh((((((((-1.0*((((data["f_0_23"]) + ((((np.minimum(((data["f_0_3"])), ((data["f_135000_71"])))) > (data["f_90000_75"]))*1.))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0499804616*np.tanh((((((((data["f_105000_19"]) < (data["f_30000_7"]))*1.)) - ((((data["f_120000_23"]) > (data["f_135000_55"]))*1.)))) - ((((data["f_135000_71"]) < (data["f_15000_31"]))*1.)))) +
            0.0500000007*np.tanh((((((((((((data["f_90000_79"]) > (((data["f_75000_51"]) / 2.0)))*1.)) * 2.0)) - ((((data["f_75000_59"]) < (data["f_60000_23"]))*1.)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_15000_87"]) > (data["f_30000_87"]))*1.)) + ((((data["f_75000_63"]) > (data["f_15000_87"]))*1.)))) - ((((data["f_60000_23"]) < (data["f_75000_63"]))*1.)))) +
            0.0499804616*np.tanh(((((data["f_60000_55"]) - ((((data["f_0_39"]) > (data["f_45000_31"]))*1.)))) - ((((np.minimum(((data["f_60000_11"])), ((data["f_0_27"])))) > (data["f_60000_35"]))*1.)))) +
            0.0499804616*np.tanh((((((data["f_15000_71"]) > (np.maximum(((data["f_75000_19"])), ((((data["f_120000_71"]) * 2.0))))))*1.)) - ((((data["f_105000_11"]) > (((data["f_60000_19"]) / 2.0)))*1.)))) +
            0.0500000007*np.tanh(((data["f_0_83"]) + (((((0.4777779877) - ((((data["f_135000_87"]) < (data["f_0_83"]))*1.)))) - ((((data["f_135000_83"]) < (data["f_0_83"]))*1.)))))) +
            0.0500000007*np.tanh((((((((((data["f_75000_7"]) < (data["f_90000_27"]))*1.)) - ((((((data["f_15000_59"]) < (((data["f_90000_27"]) * 2.0)))*1.)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0500000007*np.tanh((((((((data["f_15000_51"]) > (data["f_90000_63"]))*1.)) - ((((data["f_105000_55"]) < (data["f_0_63"]))*1.)))) - ((((data["f_0_63"]) < (data["f_15000_51"]))*1.)))) +
            0.0499804616*np.tanh((((data["f_60000_19"]) > (np.where(data["f_15000_67"]>0, data["f_30000_39"], np.minimum(((data["f_0_3"])), ((np.where(data["f_60000_79"]>0, data["f_30000_47"], data["f_60000_75"] )))) )))*1.)) +
            0.0500000007*np.tanh(((((((data["f_60000_23"]) - (data["f_30000_0"]))) - (np.where(data["f_135000_79"]>0, data["f_135000_63"], (((data["f_60000_19"]) > (data["max"]))*1.) )))) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_90000_79"]) < (np.minimum(((data["f_105000_31"])), ((np.minimum((((-1.0*((data["f_135000_83"]))))), ((data["f_60000_3"]))))))))*1.)>0, (6.0), data["f_60000_27"] )) +
            0.0499804616*np.tanh((((((data["f_75000_19"]) > (data["f_45000_43"]))*1.)) - (((np.where(data["f_105000_67"]<0, (((data["f_75000_19"]) < (data["f_90000_63"]))*1.), data["f_15000_79"] )) * 2.0)))) +
            0.0500000007*np.tanh((((((data["f_30000_47"]) > (np.where(data["f_135000_75"]<0, data["f_0_71"], np.where(data["f_0_71"]>0, ((data["f_30000_47"]) * 2.0), data["f_60000_63"] ) )))*1.)) * 2.0)) +
            0.0500000007*np.tanh(np.where((((data["f_75000_23"]) > (data["f_90000_59"]))*1.)>0, -3.0, (((data["f_15000_0"]) > (np.where(data["f_45000_47"]>0, -3.0, data["f_0_3"] )))*1.) )) +
            0.0500000007*np.tanh(((np.sin(((((((-1.0*(((((data["f_135000_27"]) < (data["f_120000_47"]))*1.))))) - ((((data["f_90000_39"]) > (data["f_105000_83"]))*1.)))) * 2.0)))) * 2.0)) +
            0.0500000007*np.tanh(np.where(np.where(data["f_60000_67"]>0, data["f_105000_75"], (((data["f_90000_23"]) < (data["f_45000_43"]))*1.) )>0, -2.0, (((data["f_60000_67"]) > (data["f_60000_19"]))*1.) )))


In [ ]:
x = GP(psdata)
score = mean_absolute_error(psdata.target, x)
print(f'Score: {score:0.5f}')